

# transformers + deepspeed CLI



In [2]:
# Free colab seems to give different amount of general RAM to different users or even the same users at different times.

!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       785Mi        80Gi       1.0Mi       2.5Gi        81Gi
Swap:             0B          0B          0B


In [3]:
# check which nvidia drivers and cuda version is running

!nvidia-smi

Mon Feb 12 01:21:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [20]:
# Need ninja when build deepspeed from source code

!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.6 MB/s eta 0:00:00


In [16]:
# 源代码安装 DeepSpeed
# 根据你的 GPU 实际情况（查看方法见前一页），设置参数 TORCH_CUDA_ARCH_LIST；
# 如果你需要使用 NVMe Offload，设置参数  DS_BUILD_UTILS=1；
# 如果你需要使用 CPU Offload 优化器参数，设置参数 DS_BUILD_CPU_ADAM=1；
!git clone https://github.com/microsoft/DeepSpeed/

Cloning into 'DeepSpeed'...
remote: Enumerating objects: 42409, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 42409 (delta 23), reused 33 (delta 17), pack-reused 42362
Receiving objects: 100% (42409/42409), 202.05 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (30590/30590), done.
Updating files: 100% (1499/1499), done.


In [21]:
%%bash


cd DeepSpeed
rm -rf build
TORCH_CUDA_ARCH_LIST="7.5" DS_BUILD_CPU_ADAM=1 DS_BUILD_UTILS=1 pip install -e . \
--global-option="build_ext" --global-option="-j8" --no-cache -v \
--disable-pip-version-check 2>&1 | tee build.log


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Obtaining file:///content/DeepSpeed
  Preparing metadata (setup.py): started
  Running command python setup.py egg_info
  [2024-02-12 01:44:47,105] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
  [2024-02-12 01:44:47,200] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
  DS_BUILD_OPS=0
   [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
   [WARNING]  async_io: please install the libaio-dev package with apt
   [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be

In [12]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install -e .
pip install -r examples/pytorch/translation/requirements.txt

Obtaining file:///content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for transformers (pyproject.toml): started
  Building editable for transformers (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.38.0.dev0-0.editable-py3-none-any.whl size=41785 sha256=d2c503075d6ad3300a366c2d3f3850191cedea581570727e8116cf5c7e77f27b
  Stored in directory: /tmp/pip-ephem-wheel-cache-d2n645ct/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully b

Cloning into 'transformers'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
%%bash

# DeepSpeed ZeRO-2 模式单 GPU 训练翻译模型（T5-Small）
cd transformers; export BS=16; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero2.json \
--model_name_or_path t5-small --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro


[2024-02-12 01:47:05,129] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-12 01:47:06,352] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2024-02-12 01:47:06,353] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None examples/pytorch/translation/run_translation.py --deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero2.json --model_name_or_path t5-small --per_device_train_batch_size 1 --output_dir output_dir --overwrite_output_dir --fp16 --do_train --max_train_samples 500 --num_train_ep

Generating dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea)
Dataset not on Hf google storage. Downloading and preparing it from source
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37.incomplete
storing hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet in cache at /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
creating metadata file for /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/validation/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/da

In [24]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-3B）
%%bash

cd transformers; export BS=20; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-3b  \
--per_device_train_batch_size $BS \
--per_device_eval_batch_size $BS \
--output_dir output_dir --overwrite_output_dir \
--do_train \
--do_eval \
--max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro \
--gradient_accumulation_steps 8 \
--fp16

[2024-02-12 01:50:17,006] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-12 01:50:18,222] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2024-02-12 01:50:18,222] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None examples/pytorch/translation/run_translation.py --deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json --model_name_or_path t5-3b --per_device_train_batch_size 20 --per_device_eval_batch_size 20 --output_dir output_dir --overwrite_output_dir --do_train --do_eval --max

In [30]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-3B）
%%bash

cd transformers; export BS=20; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-3b  \
--per_device_train_batch_size $BS \
--per_device_eval_batch_size $BS \
--output_dir /content/drive/MyDrive/models/t5-3b/ --overwrite_output_dir \
--do_train \
--do_eval \
--max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro \
--gradient_accumulation_steps 8 \
--fp16

[2024-02-12 03:14:22,662] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-12 03:14:25,204] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2024-02-12 03:14:25,204] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None examples/pytorch/translation/run_translation.py --deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json --model_name_or_path t5-3b --per_device_train_batch_size 20 --per_device_eval_batch_size 20 --output_dir /content/drive/MyDrive/models/ --overwrite_output_dir --do_t

Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea
Found cached dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea)
Loading Dataset info from /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea
[INFO|configuration_utils.py:729] 2024-02-12 03:14:42,176 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/config.json
[INFO|configuration_utils.py:792] 2024-02-12 03:14:42,182 >> Model config T5Config {
  "_name_or_path": "t5-3b",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward

In [ ]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-11B）
%%bash

cd transformers; export BS=20; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_flan_t5_z3_offload.json \
--model_name_or_path t5-11b  \
--per_device_train_batch_size $BS \
--per_device_eval_batch_size $BS \
--output_dir /content/drive/MyDrive/models/t5-11b/ --overwrite_output_dir \
--do_train \
--do_eval \
--max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro \
--gradient_accumulation_steps 8 \
--bf16

In [31]:
%%bash

cd transformers; export BS=16; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-11b  \
--output_dir /content/drive/MyDrive/models/t5-11b/ --adam_eps 1e-06 --evaluation_strategy=steps \
--do_train --do_eval --label_smoothing 0.1 --learning_rate 3e-5  \
--max_source_length 128 --max_target_length 128 --num_train_epochs 1 --overwrite_output_dir  \
--per_device_train_batch_size $BS --per_device_eval_batch_size $BS --predict_with_generate --sortish_sampler \
--val_max_target_length 128 --warmup_steps 500 --max_train_samples 2000 --max_eval_samples 500 \
--dataset_name wmt16 --dataset_config ro-en --source_lang en --target_lang ro \
--source_lang en --target_lang ro \
--gradient_accumulation_steps 8 \
--fp16




Process is interrupted.
